In [2]:
import pandas as pd
import numpy as np
import setuptools
import openml
from sklearn.linear_model import LogisticRegression 
import lightgbm as lgbm
import optuna
from scipy.spatial.distance import mahalanobis
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process.kernels import Matern
from engression import engression, engression_bagged
import torch
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import mahalanobis
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from rtdl_revisiting_models import MLP, ResNet, FTTransformer
import random
import gpytorch
import tqdm.auto as tqdm
import os
from pygam import LogisticGAM, s
import torch
from torch import nn
from torch.optim import Adam
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder 
from utils import EarlyStopping, train, train_trans, train_no_early_stopping, train_trans_no_early_stopping
from torch.utils.data import TensorDataset, DataLoader
import re
import shutil
import gpboost as gpb

# Create the checkpoint directory if it doesn't exist
if os.path.exists('CHECKPOINTS/MAHALANOBIS'):
    shutil.rmtree('CHECKPOINTS/MAHALANOBIS')
os.makedirs('CHECKPOINTS/MAHALANOBIS')

#SUITE_ID = 336 # Regression on numerical features
SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite

task_id=361055

# Set the random seed for reproducibility
N_TRIALS=100
N_SAMPLES=100
PATIENCE=40
N_EPOCHS=1000
GP_ITERATIONS=1000
BATCH_SIZE=1024
seed=10
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)

print(f"Task {task_id}")

CHECKPOINT_PATH = f'CHECKPOINTS/MAHALANOBIS/task_{task_id}.pt'

task = openml.tasks.get_task(task_id)  # download the OpenML task
dataset = task.get_dataset()

X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute)

if len(X) > 15000:
    indices = np.random.choice(X.index, size=15000, replace=False)
    X = X.iloc[indices,]
    y = y[indices]

# Remove categorical columns with more than 20 unique values and non-categorical columns with less than 10 unique values
# Remove non-categorical columns with more than 70% of the data in one category from X_clean
for col in [attribute for attribute, indicator in zip(attribute_names, categorical_indicator) if indicator]:
    if len(X[col].unique()) > 20:
        X = X.drop(col, axis=1)

X_clean=X.copy()
for col in [attribute for attribute, indicator in zip(attribute_names, categorical_indicator) if not indicator]:
    if len(X[col].unique()) < 10:
        X = X.drop(col, axis=1)
        X_clean = X_clean.drop(col, axis=1)
    elif X[col].value_counts(normalize=True).max() > 0.7:
        X_clean = X_clean.drop(col, axis=1)

# Find features with absolute correlation > 0.9
corr_matrix = X_clean.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
high_corr_features = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]

# Drop one of the highly correlated features from X_clean
X_clean = X_clean.drop(high_corr_features, axis=1)

# Rename columns to avoid problems with LGBM
X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

# Transform y to int type, to then be able to apply BCEWithLogitsLoss
# Create a label encoder
le = LabelEncoder()
# Fit the label encoder and transform y to get binary labels
y_encoded = le.fit_transform(y)
# Convert the result back to a pandas Series
y = pd.Series(y_encoded, index=y.index)

# calculate the mean and covariance matrix of the dataset
mean = np.mean(X_clean, axis=0)
cov = np.cov(X_clean.T)

# calculate the Mahalanobis distance for each data point
mahalanobis_dist = [mahalanobis(x, mean, np.linalg.inv(cov)) for x in X_clean.values]

mahalanobis_dist=pd.Series(mahalanobis_dist,index=X_clean.index)
far_index=mahalanobis_dist.index[np.where(mahalanobis_dist>=np.quantile(mahalanobis_dist,0.8))[0]]
close_index=mahalanobis_dist.index[np.where(mahalanobis_dist<np.quantile(mahalanobis_dist,0.8))[0]]

X_train_clean = X_clean.loc[close_index,:]
X_train = X.loc[close_index,:]
X_test = X.loc[far_index,:]
y_train = y.loc[close_index]
y_test = y.loc[far_index]

mean = np.mean(X_train_clean, axis=0)
cov = np.cov(X_train_clean.T)

# calculate the Mahalanobis distance for each data point
mahalanobis_dist_ = [mahalanobis(x, mean, np.linalg.inv(cov)) for x in X_train_clean.values]

mahalanobis_dist_=pd.Series(mahalanobis_dist_,index=X_train_clean.index)
far_index_=mahalanobis_dist_.index[np.where(mahalanobis_dist_>=np.quantile(mahalanobis_dist_,0.8))[0]]
close_index_=mahalanobis_dist_.index[np.where(mahalanobis_dist_<np.quantile(mahalanobis_dist_,0.8))[0]]

X_train_ = X_train.loc[close_index_,:]
X_val = X_train.loc[far_index_,:]
y_train_ = y_train.loc[close_index_]
y_val = y_train.loc[far_index_]


# Standardize the data
mean_X_train_ = np.mean(X_train_, axis=0)
std_X_train_ = np.std(X_train_, axis=0)
X_train_ = (X_train_ - mean_X_train_) / std_X_train_
X_val = (X_val - mean_X_train_) / std_X_train_

mean_X_train = np.mean(X_train, axis=0)
std_X_train = np.std(X_train, axis=0)
X_train = (X_train - mean_X_train) / std_X_train
X_test = (X_test - mean_X_train) / std_X_train


# Convert data to PyTorch tensors
X_train__tensor = torch.tensor(X_train_.values, dtype=torch.float32)
y_train__tensor = torch.tensor(y_train_.values, dtype=torch.float32)
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Convert to use GPU if available
if torch.cuda.is_available():
    X_train__tensor = X_train__tensor.cuda()
    y_train__tensor = y_train__tensor.cuda()
    X_train_tensor = X_train_tensor.cuda()
    y_train_tensor = y_train_tensor.cuda()
    X_val_tensor = X_val_tensor.cuda()
    y_val_tensor = y_val_tensor.cuda()
    X_test_tensor = X_test_tensor.cuda()
    y_test_tensor = y_test_tensor.cuda()

# Create flattened versions of the data
y_val_np = y_val.values.flatten()
y_test_np = y_test.values.flatten()

# Create TensorDatasets for training and validation sets
train__dataset = TensorDataset(X_train__tensor, y_train__tensor)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders for training and validation sets
train__loader = DataLoader(train__dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

d_out = 1  
d_in=X_train_.shape[1]

Task 361055


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


In [5]:
intercept_train=np.ones(X_train.shape[0])
intercept_test=np.ones(X_test.shape[0])
gp_model = gpb.GPModel(gp_coords=X_train, cov_function="gaussian_ard", likelihood="bernoulli_logit", gp_approx="vecchia", matrix_inversion_method="iterative")
gp_model.fit(y=y_train, X=intercept_train, params={"trace": True})

[GPBoost] [Info] Starting nearest neighbor search for Vecchia approximation
[GPBoost] [Info] Nearest neighbors for Vecchia approximation found
[GPBoost] [Debug] GPModel: initial parameters: 
[GPBoost] [Debug] cov_pars[0]: 1
[GPBoost] [Debug] cov_pars[1]: 0.0635703
[GPBoost] [Debug] cov_pars[2]: 0.326035
[GPBoost] [Debug] cov_pars[3]: 0.236687
[GPBoost] [Debug] cov_pars[4]: 0.0387337
[GPBoost] [Debug] cov_pars[5]: 0.324137
[GPBoost] [Debug] cov_pars[6]: 0.342771
[GPBoost] [Debug] cov_pars[7]: 0.209771
[GPBoost] [Debug] cov_pars[8]: 0.314717
[GPBoost] [Debug] cov_pars[9]: 0.194184
[GPBoost] [Debug] beta[0]: 0.0406723
[GPBoost] [Debug] Initial approximate negative marginal log-likelihood: 7936.8
[GPBoost] [Debug] Nearest neighbors redetermined after iteration number 1 
[GPBoost] [Debug] GPModel: parameters after optimization iteration number 1: 
[GPBoost] [Debug] cov_pars[0]: 1.16036
[GPBoost] [Debug] cov_pars[1]: 0.0684712
[GPBoost] [Debug] cov_pars[2]: 0.378035
[GPBoost] [Debug] cov_par

In [7]:
pred_resp = gp_model.predict(gp_coords_pred=X_test, X_pred=intercept_test, predict_var=False, predict_response=True)

In [14]:
pred_resp=pred_resp['mu']

In [15]:
log_loss(y_test, pred_resp)

0.5718786704763885

In [16]:
from sklearn.metrics import accuracy_score

In [19]:
pred_resp = np.where(pred_resp >= 0.5, 1, 0)

In [20]:
accuracy_score(y_test, pred_resp)

0.711